In this notebook, we will have Gaussian PSF and Gaussian galaxies, and characterize the noise and bias in the GAaP measurements in increasingly realistic scenarios.

In [1]:
import lsst.meas.extensions.gaap
import lsst.utils.tests
import lsst.meas.base as measBase
import lsst.meas.base.tests
import lsst.daf.base
import lsst.afw.geom as afwGeom

In [2]:
import lsst.ip.diffim as ip_diffim
ip_diffim.modelPsfMatch.ModelPsfMatchTask.run?

In [24]:
class CharacterizeGaap(lsst.meas.base.tests.AlgorithmTestCase, lsst.utils.tests.TestCase):
    
    def setUp(self, noise):
        self.center = lsst.geom.Point2D(100, 700)
        self.bbox = lsst.geom.Box2I(lsst.geom.Point2I(-20, -30), lsst.geom.Extent2I(240, 1600))
        self.dataset = lsst.meas.base.tests.TestDataset(self.bbox)
        # Insert multiple sources here
        self.dataset.addSource(1000, self.center + lsst.geom.Extent2I(0, 100), afwGeom.Quadrupole(9,9,4))
        
        schema = lsst.meas.base.tests.TestDataset.makeMinimalSchema()
        detectConfig = lsst.meas.algorithms.SourceDetectionConfig()
        detectConfig.returnOriginalFootprints = False
        detectConfig.thresholdValue = 10
        detectTask = lsst.meas.algorithms.SourceDetectionTask(config=detectConfig, schema=schema)
        deblendTask = lsst.meas.deblender.SourceDeblendTask(schema=schema)
        
        table = lsst.afw.table.SourceTable.make(schema)
        exposure, catalog = self.dataset.realize(noise, schema)
        detectResult = detectTask.run(table, exposure)
        catalog = detectResult.sources
        
        deblendTask.run(exposure, catalog)
        
    def tearDown(self):
        del self.center
        del self.bbox
        del self.dataset
        
    def test1(self, noise):
        algName = "ext_gaap_GaapFlux"
        dependencies = ("base_SdssShape",)
        algMetadata = lsst.daf.base.PropertyList()
        sfmConfig = self.makeSingleFrameMeasurementConfig(algName, dependencies=dependencies)
        sfmConfig.plugins[algName].doPsfPhotometry = True
        sfmTask = self.makeSingleFrameMeasurementTask(config=sfmConfig, algMetadata=algMetadata)
        # exposure, catalog = self.dataset.realize(0.0, sfmTask.schema)
        sfmTask.run(catalog, exposure)
        return exposure, catalog
        

In [25]:
instance = CharacterizeGaap()
instance.setUp(10.0)
exp, cat = instance.test1(10.0)

InvalidParameterError: 
  File "src/table/Schema.cc", line 394, in lsst::afw::table::Key<U> lsst::afw::table::detail::SchemaImpl::addFieldImpl(int, int, const lsst::afw::table::Field<T>&, bool) [with T = int]
    Field with name 'deblend_nChild' already present in schema. {0}
lsst::pex::exceptions::InvalidParameterError: 'Field with name 'deblend_nChild' already present in schema.'


In [17]:
cat.asAstropy()

id,coord_ra,coord_dec,parent,deblend_nChild,truth_instFlux,slot_Centroid_instFlux,slot_Shape_instFlux,truth_x,slot_Centroid_x,slot_Shape_x,truth_y,slot_Centroid_y,slot_Shape_y,truth_xErr,slot_Centroid_xErr,slot_Shape_xErr,truth_yErr,slot_Centroid_yErr,slot_Shape_yErr,truth_x_y_Cov,slot_Centroid_x_y_Cov,slot_Shape_x_y_Cov,truth_flag,base_SdssShape_flag_badCentroid,slot_Centroid_flag,slot_Shape_flag,truth_xx,slot_Centroid_xx,slot_Shape_xx,truth_yy,slot_Centroid_yy,slot_Shape_yy,truth_xy,slot_Centroid_xy,slot_Shape_xy,truth_isStar,slot_Centroid_isStar,slot_Shape_isStar,base_SdssShape_xx,base_SdssShape_yy,base_SdssShape_xy,base_SdssShape_xxErr,base_SdssShape_yyErr,base_SdssShape_xyErr,base_SdssShape_x,base_SdssShape_y,base_SdssShape_instFlux,base_SdssShape_instFluxErr,base_SdssShape_psf_xx,slot_PsfShape_xx,base_SdssShape_psf_yy,slot_PsfShape_yy,base_SdssShape_psf_xy,slot_PsfShape_xy,base_SdssShape_instFlux_xx_Cov,base_SdssShape_instFlux_yy_Cov,base_SdssShape_instFlux_xy_Cov,base_SdssShape_flag,base_SdssShape_flag_unweightedBad,base_SdssShape_flag_unweighted,base_SdssShape_flag_shift,base_SdssShape_flag_maxIter,base_SdssShape_flag_psf,ext_gaap_GaapFlux_1_15x_0_7_instFlux,ext_gaap_GaapFlux_1_15x_0_7_instFluxErr,ext_gaap_GaapFlux_1_15x_1_0_instFlux,ext_gaap_GaapFlux_1_15x_1_0_instFluxErr,ext_gaap_GaapFlux_1_15x_PsfFlux_instFlux,ext_gaap_GaapFlux_1_15x_PsfFlux_instFluxErr,ext_gaap_GaapFlux_OptimalShape_xx,ext_gaap_GaapFlux_OptimalShape_yy,ext_gaap_GaapFlux_OptimalShape_xy,ext_gaap_GaapFlux_1_15x_Optimal_instFlux,ext_gaap_GaapFlux_1_15x_Optimal_instFluxErr,ext_gaap_GaapFlux_1_15x_0_7_flag_bigPsf,ext_gaap_GaapFlux_1_15x_0_7_flag,ext_gaap_GaapFlux_1_15x_1_0_flag_bigPsf,ext_gaap_GaapFlux_1_15x_1_0_flag,ext_gaap_GaapFlux_1_15x_PsfFlux_flag,ext_gaap_GaapFlux_1_15x_Optimal_flag_bigPsf,ext_gaap_GaapFlux_1_15x_Optimal_flag,ext_gaap_GaapFlux_1_15x_flag_gaussianization,ext_gaap_GaapFlux_flag_edge,ext_gaap_GaapFlux_flag
,rad,rad,,,ct,ct,ct,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix2,pix2,pix2,,,,,pix2,pix2,pix2,pix2,pix2,pix2,pix2,pix2,pix2,,,,pix2,pix2,pix2,pix2,pix2,pix2,pix,pix,ct,ct,pix2,pix2,pix2,pix2,pix2,pix2,ct pix2,ct pix2,ct pix2,,,,,,,ct,ct,ct,ct,ct,ct,pix2,pix2,pix2,ct,ct,,,,,,,,,,
int64,float64,float64,int64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,bool,bool,bool,bool,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
1,nan,nan,0,0,1000.0,1000.0,1000.0,100.0,100.0,100.0,800.0,800.0,800.0,nan,nan,nan,nan,nan,nan,-0.037866678,-0.037866678,-0.037866678,False,False,False,False,13.0,13.0,13.0,13.0,13.0,13.0,4.0,4.0,4.0,False,False,False,12.999725116641704,12.999725116641704,3.999917393392555,nan,nan,nan,100.0000009769953,800.000000410285,999.9886565039415,nan,4.0,4.0,4.0,4.0,0.0,0.0,nan,nan,nan,False,False,False,False,False,False,586.9424097320937,0.0,740.4359517013024,0.0,551.9648998592165,0.0,17.0,17.0,4.0,667.8888063005387,0.0,False,False,False,False,False,False,False,False,False,False
